In [ ]:
!pip install requests twilio schedule

In [3]:
from twilio.rest import Client
import requests
import schedule
import time
import json

In [16]:
r = requests.get('https://api.github.com/events')


In [9]:
r = requests.post('https://httpbin.org/post', data={'key': 'value'})

In [17]:
payload = {'key1': 'value1', 'key2': 'value2'}
r = requests.get("https://httpbin.org/get", params=payload)

In [18]:
print(r.url)

https://httpbin.org/get?key1=value1&key2=value2


In [19]:
r = requests.get('https://api.github.com/events')
r.text

'[{"id":"42576004604","type":"PushEvent","actor":{"id":41898282,"login":"github-actions[bot]","display_login":"github-actions","gravatar_id":"","url":"https://api.github.com/users/github-actions[bot]","avatar_url":"https://avatars.githubusercontent.com/u/41898282?"},"repo":{"id":671292637,"name":"dfef22/to-keep-green","url":"https://api.github.com/repos/dfef22/to-keep-green"},"payload":{"repository_id":671292637,"push_id":20572347298,"size":1,"distinct_size":1,"ref":"refs/heads/main","head":"6727785773e07f368583e7a7f7b1c09f1d3d57ca","before":"92747de4a72b0b196bc4224a806695aa79f64493","commits":[{"sha":"6727785773e07f368583e7a7f7b1c09f1d3d57ca","author":{"email":"dfef22@163.com","name":"dfef22"},"message":"a commit a day keeps your girlfriend away","distinct":true,"url":"https://api.github.com/repos/dfef22/to-keep-green/commits/6727785773e07f368583e7a7f7b1c09f1d3d57ca"}]},"public":true,"created_at":"2024-10-06T03:23:43Z"},{"id":"42576004602","type":"PushEvent","actor":{"id":173535095,"l

In [26]:
def get_gridpoint_forecast(lat, lon):
    """
    Fetch the gridpoint forecast for the given latitude and longitude.
    """
    base_url = 'https://api.weather.gov/points'
    point_url = f"{base_url}/{lat},{lon}"
    headers = {
        'User-Agent': '(rainy_season_alerts_app, little.ge@yahoo.com)'
    }

    # Fetch grid data using the /points endpoint
    try:
        point_response = requests.get(point_url, headers=headers)
        point_response.raise_for_status()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        return None
    except Exception as err:
        print(f"Error occurred: {err}")
        return None

    point_data = point_response.json()
    # print("this is point data: \n", json.dumps(point_data, indent=2))

    # Retrieve forecast URL from the grid data
    forecast_url = point_data['properties']['forecast']

    # Fetch the forecast data
    try:
        forecast_response = requests.get(forecast_url, headers=headers)
        forecast_response.raise_for_status()
        # print("this is forecast data: \n", json.dumps(forecast_response.json(), indent=2))
        return forecast_response.json()['properties']['periods']
        
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except Exception as err:
        print(f"Error occurred: {err}")

    return None


In [27]:
latitude = 30.4383
longitude = -84.2807

forecast_data = get_gridpoint_forecast(latitude, longitude)

if forecast_data:
    for period in forecast_data:
        print(f"{period['name']}: {period['detailedForecast']}")


Today: A chance of showers and thunderstorms. Mostly cloudy. High near 80, with temperatures falling to around 77 in the afternoon. East northeast wind around 10 mph. Chance of precipitation is 50%. New rainfall amounts less than a tenth of an inch possible.
Tonight: A chance of showers and thunderstorms before 7pm, then a slight chance of showers and thunderstorms between 7pm and 8pm. Mostly cloudy, with a low around 69. Northeast wind 5 to 10 mph. Chance of precipitation is 30%. New rainfall amounts less than a tenth of an inch possible.
Monday: Sunny. High near 86, with temperatures falling to around 83 in the afternoon. Northeast wind 5 to 10 mph.
Monday Night: Mostly clear, with a low around 66. Northeast wind 0 to 5 mph.
Tuesday: Mostly sunny, with a high near 84. Northeast wind 5 to 10 mph, with gusts as high as 25 mph.
Tuesday Night: Mostly cloudy, with a low around 67.
Wednesday: Tropical storm conditions possible. A chance of rain showers after 8am. Partly sunny, with a high 

In [28]:
def check_rain_conditions(forecast_data):
    for period in forecast_data:
        if 'precipitation' in period['detailedForecast'].lower():
            chance_of_rain = period.get('probabilityOfPrecipitation', {}).get('value', 0)
            rainfall_amount = period.get('quantitativePrecipitation', {}).get('value', 0)
            
            if chance_of_rain >= 50 or rainfall_amount >= 0.10:
                return True, period['name'], chance_of_rain, rainfall_amount
    return False, None, 0, 0


In [29]:
check_rain_conditions(forecast_data)

(True, 'Today', 50, 0)

In [ ]:
def send_alert(to_phone_number, message_body):
    # Twilio credentials
    account_sid = 'your_account_sid'
    auth_token = 'your_auth_token'
    twilio_phone_number = 'your_twilio_phone_number'
    
    client = Client(account_sid, auth_token)
    
    message = client.messages.create(
        body=message_body,
        from_=twilio_phone_number,
        to=to_phone_number
    )
    
    print(f"Message sent: {message.sid}")


In [ ]:
def run_daily_check(zip_code, phone_number):
    forecast_data = get_weather_forecast(zip_code)
    
    if forecast_data:
        condition_met, period_name, chance_of_rain, rainfall_amount = check_rain_conditions(forecast_data)
        
        if condition_met:
            message = f"Reminder: {period_name} has a {chance_of_rain}% chance of rain with {rainfall_amount} inches of expected rainfall."
            send_alert(phone_number, message)
        else:
            print("No rain conditions met for today.")
    else:
        print("Unable to fetch forecast data.")

# Schedule the task to run every day at a specific time (e.g., 7 AM)
schedule.every().day.at("07:00").do(run_daily_check, zip_code='90210', phone_number='+1234567890')

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(60)  # Check every minute if the task is due


### config.json

In [ ]:
{
    "zip_code": "90210",
    "phone_number": "+1234567890"
}


In [ ]:
def load_config():
    with open('config.json') as config_file:
        return json.load(config_file)

config = load_config()

schedule.every().day.at("07:00").do(run_daily_check, zip_code=config['zip_code'], phone_number=config['phone_number'])
